In [5]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

from keras.utils.np_utils import to_categorical

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [6]:
batch_size=64

In [7]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

X_train and X_test need to be in dimensions (1000, 1, 28, 28)
Keras expects array

In [8]:
X_train_resize = X_train[:, np.newaxis, :, :]
X_test_resize = X_test[:, np.newaxis, :, :]
# print(X_train.shape)
# print(X_train_resize.shape)
# print(X_train[0])
# print(X_train_resize[0])

y's need to be one hot encoded

In [9]:
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

remember to normalize x's

In [10]:
x_mean = np.mean(X_train_resize, dtype=np.float32)
x_std = np.std(X_train_resize, dtype=np.float32)
def norm_input(x):
    return (x - x_mean) / x_std
    

In [39]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train_resize, y_train_hot)
test_batches = gen.flow(X_test_resize, y_test_hot)


In [72]:
# X_train_resize.shape[1:]
# (3,) + (28, 28)
X_train.dtype
test = norm_input(X_train_resize)
test.dtype
# print(X_train_resize.shape[1:])
test.ndim

4

## Linear Model

In [100]:
def get_lin_model(norm_input_function):
    input_shape = X_train_resize.shape[1:]
    return Sequential(layers=[
        Lambda(norm_input_function, input_shape=input_shape, output_shape=input_shape),
        Flatten(),
        Dense(10, activation='softmax')
    ])

In [101]:
lm = get_lin_model(norm_input)
lm.compile(Adam(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [102]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_16 (Lambda)               (None, 1, 28, 28)     0           lambda_input_15[0][0]            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 784)           0           lambda_16[0][0]                  
____________________________________________________________________________________________________
dense_16 (Dense)                 (None, 10)            7850        flatten_2[0][0]                  
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
____________________________________________________________________________________________________


run fit (epoch 1)

In [105]:
lm.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1, 
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 1s - loss: 1.6140 - acc: 0.5000 - val_loss: 1.2176 - val_acc: 0.6151


In [106]:
lm.optimizer.lr = 0.1

In [108]:
lm.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1, 
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 1s - loss: 0.7869 - acc: 0.7188 - val_loss: 0.9233 - val_acc: 0.7308


run fit (epoch 1)

In [109]:
lm.optimizer.lr = 0.01

In [110]:
lm.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1, 
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 1s - loss: 0.7225 - acc: 0.7969 - val_loss: 1.0506 - val_acc: 0.7187


run fit (epoch 4)

## Single Dense Layer

In [113]:
def get_fc_model():
    input_shape = X_train_resize.shape[1:]
    model = Sequential(layers=[
        Lambda(norm_input, input_shape=input_shape, output_shape=input_shape),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(10, activation='softmax')
    ])
    #Sequential
    #Dense - 512
    return model

In [118]:
fc = get_fc_model()
fc.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

run fit (epoch 1)

In [119]:
fc.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 1s - loss: 2.5918 - acc: 0.1406 - val_loss: 2.0577 - val_acc: 0.3323


In [120]:
fc.optimizer.lr = 0.1

In [121]:
fc.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=4,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/4
64/64 [==============================] - 1s - loss: 2.1412 - acc: 0.2969 - val_loss: 1.4717 - val_acc: 0.5028
Epoch 2/4
64/64 [==============================] - 1s - loss: 1.5354 - acc: 0.5469 - val_loss: 1.1167 - val_acc: 0.6543
Epoch 3/4
64/64 [==============================] - 1s - loss: 0.9864 - acc: 0.6875 - val_loss: 0.9290 - val_acc: 0.7009
Epoch 4/4
64/64 [==============================] - 1s - loss: 0.9427 - acc: 0.7812 - val_loss: 0.8100 - val_acc: 0.7315


run fit (epoch 4)

In [122]:
fc.optimizer.lr = 0.01

In [123]:
fc.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=4,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/4
64/64 [==============================] - 1s - loss: 0.9236 - acc: 0.6875 - val_loss: 0.6765 - val_acc: 0.7843
Epoch 2/4
64/64 [==============================] - 1s - loss: 0.7349 - acc: 0.7969 - val_loss: 0.6010 - val_acc: 0.8153
Epoch 3/4
64/64 [==============================] - 1s - loss: 0.7384 - acc: 0.7812 - val_loss: 0.6098 - val_acc: 0.8055
Epoch 4/4
64/64 [==============================] - 1s - loss: 0.4997 - acc: 0.7812 - val_loss: 0.6151 - val_acc: 0.8122


run fit (epoch 4)

## Basic 'VGG-style' CNN

In [47]:
def get_model():
    x_shape = X_train_resize.shape[1:]
    model = Sequential(layers=[
        Lambda(norm_input, output_shape=x_shape, input_shape=x_shape),
        Convolution2D(32, 3, 3, subsample=(1, 1), activation='relu'),
        Convolution2D(32, 3, 3, subsample=(1, 1), activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Convolution2D(64, 3, 3, subsample=(1, 1), activation='relu'),
        Convolution2D(64, 3, 3, subsample=(1, 1), activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
#         Dense(4096, activation='relu')
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(10, activation='softmax')
    ])
    return model
    

In [48]:
model = get_model()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

run fit (epoch 1)

In [49]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 2s - loss: 2.2808 - acc: 0.1719 - val_loss: 2.2356 - val_acc: 0.1725


In [ ]:
model.optimizer.lr = 0.1

run fit (epoch 1)

In [51]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=4,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/4
64/64 [==============================] - 2s - loss: 0.9642 - acc: 0.7500 - val_loss: 0.6965 - val_acc: 0.7882
Epoch 2/4
64/64 [==============================] - 2s - loss: 0.8888 - acc: 0.6719 - val_loss: 0.6060 - val_acc: 0.8009
Epoch 3/4
64/64 [==============================] - 2s - loss: 0.7336 - acc: 0.7656 - val_loss: 0.5097 - val_acc: 0.8398
Epoch 4/4
64/64 [==============================] - 2s - loss: 0.8539 - acc: 0.7969 - val_loss: 0.5373 - val_acc: 0.8245


In [ ]:
model.optimizer.lr = 0.01

run fit (epoch 8)

In [52]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=8,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/8
64/64 [==============================] - 3s - loss: 0.8632 - acc: 0.7656 - val_loss: 0.4514 - val_acc: 0.8537
Epoch 2/8
64/64 [==============================] - 2s - loss: 0.3445 - acc: 0.8750 - val_loss: 0.5181 - val_acc: 0.8527
Epoch 3/8
64/64 [==============================] - 2s - loss: 0.6807 - acc: 0.7969 - val_loss: 0.5267 - val_acc: 0.8496
Epoch 4/8
64/64 [==============================] - 2s - loss: 0.5108 - acc: 0.8438 - val_loss: 0.4645 - val_acc: 0.8674
Epoch 5/8
64/64 [==============================] - 2s - loss: 0.5467 - acc: 0.7812 - val_loss: 0.4019 - val_acc: 0.8856
Epoch 6/8
64/64 [==============================] - 2s - loss: 0.2366 - acc: 0.9531 - val_loss: 0.3789 - val_acc: 0.8873
Epoch 7/8
64/64 [==============================] - 2s - loss: 0.6181 - acc: 0.7812 - val_loss: 0.3621 - val_acc: 0.8825
Epoch 8/8
64/64 [==============================] - 3s - loss: 0.2848 - acc: 0.9219 - val_loss: 0.3581 - val_acc: 0.8769


## Data augmentation

In [ ]:
model = get_model()
# L C C M C C M F D D

In [62]:
# gen = image.ImageDataGenerator(rotation_range=10, zoom_range=[.9, 1.1], width_shift_range=.1, height_shift_range=.1)
gen = image.ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                               height_shift_range=0.08, zoom_range=0.08)
batches = gen.flow(X_train_resize, y_train_hot)
test_batches = gen.flow(X_test_resize, y_test_hot)

run fit (epoch 1)

In [54]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=1,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 2s - loss: 1.2523 - acc: 0.6875 - val_loss: 1.1184 - val_acc: 0.6413


In [55]:
model.optimizer.lr = 0.1

run fit (epoch 4)

In [54]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=4,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/1
64/64 [==============================] - 2s - loss: 1.2523 - acc: 0.6875 - val_loss: 1.1184 - val_acc: 0.6413


In [56]:
model.optimizer.lr = 0.01

run fit (epoch 8)

In [57]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=8,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/8
64/64 [==============================] - 2s - loss: 1.0882 - acc: 0.6562 - val_loss: 1.0707 - val_acc: 0.6463
Epoch 2/8
64/64 [==============================] - 2s - loss: 1.4595 - acc: 0.5469 - val_loss: 0.9763 - val_acc: 0.6917
Epoch 3/8
64/64 [==============================] - 2s - loss: 1.0070 - acc: 0.6719 - val_loss: 1.0287 - val_acc: 0.6974
Epoch 4/8
64/64 [==============================] - 2s - loss: 1.0822 - acc: 0.6406 - val_loss: 1.0405 - val_acc: 0.6888
Epoch 5/8
64/64 [==============================] - 2s - loss: 1.0109 - acc: 0.6406 - val_loss: 0.9187 - val_acc: 0.7260
Epoch 6/8
64/64 [==============================] - 2s - loss: 1.0123 - acc: 0.6562 - val_loss: 0.8400 - val_acc: 0.7418
Epoch 7/8
64/64 [==============================] - 2s - loss: 0.8344 - acc: 0.7500 - val_loss: 0.7837 - val_acc: 0.7469
Epoch 8/8
64/64 [==============================] - 2s - loss: 0.9048 - acc: 0.7031 - val_loss: 0.7195 - val_acc: 0.7646


In [58]:
model.optimizer.lr = 0.001

run fit (epoch 14)

In [59]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=14,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/14
64/64 [==============================] - 2s - loss: 0.6205 - acc: 0.7500 - val_loss: 0.6891 - val_acc: 0.7755
Epoch 2/14
64/64 [==============================] - 2s - loss: 0.5853 - acc: 0.8125 - val_loss: 0.6663 - val_acc: 0.7801
Epoch 3/14
64/64 [==============================] - 2s - loss: 0.9480 - acc: 0.7031 - val_loss: 0.6114 - val_acc: 0.7978
Epoch 4/14
64/64 [==============================] - 2s - loss: 0.7151 - acc: 0.7344 - val_loss: 0.5931 - val_acc: 0.8033
Epoch 5/14
64/64 [==============================] - 2s - loss: 0.4479 - acc: 0.8750 - val_loss: 0.6520 - val_acc: 0.7870
Epoch 6/14
64/64 [==============================] - 2s - loss: 0.8870 - acc: 0.7656 - val_loss: 0.6089 - val_acc: 0.8018
Epoch 7/14
64/64 [==============================] - 2s - loss: 0.5239 - acc: 0.8281 - val_loss: 0.5285 - val_acc: 0.8298
Epoch 8/14
64/64 [==============================] - 2s - loss: 0.4591 - acc: 0.8594 - val_loss: 0.5968 - val_acc: 0.8042
Epoch 9/14
64/64 [==============

In [60]:
model.optimizer.lr = 0.0001

run fit (epoch 10)

In [61]:
model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=10,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/10
64/64 [==============================] - 2s - loss: 0.5564 - acc: 0.7969 - val_loss: 0.4836 - val_acc: 0.8402
Epoch 2/10
64/64 [==============================] - 2s - loss: 0.5560 - acc: 0.8125 - val_loss: 0.4745 - val_acc: 0.8473
Epoch 3/10
64/64 [==============================] - 2s - loss: 0.5600 - acc: 0.8125 - val_loss: 0.5487 - val_acc: 0.8279
Epoch 4/10
64/64 [==============================] - 2s - loss: 0.4922 - acc: 0.8906 - val_loss: 0.5546 - val_acc: 0.8299
Epoch 5/10
64/64 [==============================] - 2s - loss: 0.8677 - acc: 0.7344 - val_loss: 0.4611 - val_acc: 0.8569
Epoch 6/10
64/64 [==============================] - 2s - loss: 0.3769 - acc: 0.8750 - val_loss: 0.4868 - val_acc: 0.8387
Epoch 7/10
64/64 [==============================] - 2s - loss: 0.5791 - acc: 0.7812 - val_loss: 0.4698 - val_acc: 0.8522
Epoch 8/10
64/64 [==============================] - 2s - loss: 0.5328 - acc: 0.7969 - val_loss: 0.4592 - val_acc: 0.8549
Epoch 9/10
64/64 [==============

## Batchnorm + data augmentation

In [ ]:
def get_model_bn():
    # L C B C M B C B C M F B D B D

run fit (epoch 1)

In [ ]:
model.optimizer.lr = 0.1

run fit (epoch 4)

In [ ]:
model.optimizer.lr = 0.01

run fit (epoch 12)

In [ ]:
model.optimizer.lr = 0.001

run fit (epoch 12)

## Batchnorm + dropout + data augmentation

In [68]:
def get_model_bn_do():
    x_shape = X_train_resize.shape[1:]
    # L C B C M B C B C M F B D B D D
    return Sequential(layers=[
        Lambda(norm_input, input_shape=x_shape, output_shape=output_shape),
        Convolution2D(32, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(32, 3, 3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        BatchNormalization(axis=1),
        Convolution2D(64, 3, 3, activation='relu'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Flatten(),
        BatchNormalization(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax'),
    ])

In [69]:
bn_model = get_model()
bn_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [70]:
bn_model.fit_generator(batches, samples_per_epoch=batch_size, nb_epoch=14,
                 validation_data=test_batches, nb_val_samples=y_test_hot.shape[0])

Epoch 1/14
64/64 [==============================] - 2s - loss: 2.3066 - acc: 0.0938 - val_loss: 2.2945 - val_acc: 0.1915
Epoch 2/14
64/64 [==============================] - 2s - loss: 2.3950 - acc: 0.1406 - val_loss: 2.2386 - val_acc: 0.3539
Epoch 3/14
64/64 [==============================] - 2s - loss: 2.2312 - acc: 0.2812 - val_loss: 2.2332 - val_acc: 0.2542
Epoch 4/14
64/64 [==============================] - 2s - loss: 2.1748 - acc: 0.2969 - val_loss: 2.1947 - val_acc: 0.2611
Epoch 5/14
64/64 [==============================] - 2s - loss: 2.1798 - acc: 0.2188 - val_loss: 2.0797 - val_acc: 0.3937
Epoch 6/14
64/64 [==============================] - 2s - loss: 2.0766 - acc: 0.3750 - val_loss: 1.8774 - val_acc: 0.4683
Epoch 7/14
64/64 [==============================] - 2s - loss: 1.9309 - acc: 0.4844 - val_loss: 1.6268 - val_acc: 0.5890
Epoch 8/14
64/64 [==============================] - 2s - loss: 1.6154 - acc: 0.5625 - val_loss: 1.3619 - val_acc: 0.5992
Epoch 9/14
64/64 [==============